In [1]:
install.packages("extRemes")
install.packages("cobs")
install.packages("moments")
install.packages("goftest")
install.packages("gbutils")
install.packages("forecast")
install.packages("reticulate")
# install.packages("parallel")  # parallel is base package so no need.

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Lmoments’, ‘distillery’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
system("git clone https://<SSH_ID>@github.com/Yash-10/arps.git")

In [3]:
setwd("/kaggle/working/arps")

In [4]:
source("standardize_periodogram.R")
source("eva_periodogram.R")

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot




In [5]:
library(parallel)
detectCores()

[1] 4

In [6]:
getFAP <- function(depth, ...) {
  result <- try(
      suppressWarnings(evd(depth=depth, ...))  # Note: mclapply will fail (and restart runtime) if the driver function raises warnings. Hence suppress them.
  )
  result <- if (inherits(result, "try-error")) c(NA, NA) else result 
  fap <- result[1]
}

algoComparison <- function(
    depths = depths,
    L = 500,
    R = 300,
    ntransits = 10,
    significanceMode = 'max',
    noiseType = 1,
    period = 2,  # days.
    duration = 2,  # hours.
    seedValue=1
) {
    limiting_depth_BLS <- NULL
    # For BLS.
    counter <- 1
    for (depth in depths) {
        fap <- getFAP(depth, L=L, R=R, period=period, duration=duration, noiseType=noiseType,
                        algo="BLS", ntransits=ntransits, significanceMode=significanceMode,
                          useStandardization=FALSE, ofac=2, res=2, useOptimalFreqSampling=TRUE,
                          checkConditions=TRUE, plot=FALSE, seedValue=seedValue
                     )
        if (fap <= 0.01 & !is.na(fap)) {
            limiting_depth_BLS <- depths[counter]
            break
        }
        counter <- counter + 1

    }
    if (is.null(limiting_depth_BLS)) {
        limiting_depth_BLS <- depths[length(depths)]
    }

    limiting_depth_TCF <- NULL
    # For TCF
    counter <- 1
    for (depth in depths) {
        fap <- getFAP(depth, L=L, R=R, period=period, duration=duration, noiseType=noiseType,
                        algo="TCF", ntransits=ntransits, significanceMode=significanceMode,
                          useStandardization=FALSE, ofac=2, res=2, useOptimalFreqSampling=TRUE,
                          checkConditions=TRUE, plot=FALSE, seedValue=seedValue
                     )
        if (fap <= 0.01 & !is.na(fap)) {
            limiting_depth_TCF <- depths[counter]
            break
        }
        counter <- counter + 1

    }
    if (is.null(limiting_depth_TCF)) {
        limiting_depth_TCF <- depths[length(depths)]
    }

    return (c(limiting_depth_BLS, limiting_depth_TCF))
}

- Make automatic saving of fap_depth `rds` files using git commands from within this notebook.
- Transit durations to experiment: [0.5, 1, 2, 3, 4, 5, 6] hours
- Periods: [1, 3, 5, 7, 9, 11, 14, 17, 20] days

```R
durations <- c(
    0.5, 1, 2, 3, 4, 5, 6
)
```

In [7]:
# depths <- seq(from = 0.005, to = 0.1, length.out=10)
# depths <- c(
#     seq(from = 0.005, to = 0.03, length.out=30),
# #     seq(from = 0.031, to = 0.1, length.out=10)
# )
depths <- seq(from = 0.005, to = 0.03, length.out=35)

limitingDepthBLSArr <- c()
limitingDepthTCFArr <- c()

duration <- 2.0
seedVals <- c(1, 5, 10, 1234, 54321, 19482)

In [1]:
period <- 0.5
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

[1] "Starting with period = 0.500000 days"


ERROR: Error in eval(expr, envir, enclos): object 'seedVals' not found


In [ ]:
period <- 1
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

In [ ]:
period <- 4
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

In [ ]:
period <- 7
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

In [ ]:
period <- 10
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

In [ ]:
period <- 13
print(sprintf("Starting with period = %f days", period))

ldb <- c()
ldt <- c()
for (seedValue in seedVals) {
    msgs <- capture.output(capture.output(
        result <- algoComparison(
        depths = depths,
        L = 500,
        R = 300,
        ntransits = 10,
        significanceMode = 'max',
        noiseType = 2,
        period = period,
        duration = duration,
        seedValue=seedValue
    ), type = "message"), type="output")
    print(cat("\t", "Limiting depths for BLS and TCF:", result[1], result[2]
    ldb <- c(ldb, result[1])
    ldt <- c(ldt, result[2])
}
ldbMean <- mean(ldb)
ldtMean <- mean(ldt)

print(cat("Averaged limiting depths across five different runs (for BLS and TCF)", ldbMean, ldtMean))

limitingDepthBLSArr <- c(limitingDepthBLSArr, ldbMean)
limitingDepthTCFArr <- c(limitingDepthTCFArr, ldtMean)

In [ ]:
print(limitingDepthBLSArr)
print(limitingDepthTCFArr)

In [ ]:
saveRDS(limitingDepthBLSArr, sprintf("limitingDepthBLSArr_duration%f_AR_ntransits10.rds", duration))
saveRDS(limitingDepthTCFArr, sprintf("limitingDepthTCFArr_duration%f_AR_ntransits10.rds", duration))

In [ ]:
# print(system(sprintf("mkdir fap_experiments/limitingDepths_duration%f_Gaussian_ntransits10", duration), intern=TRUE))

In [ ]:
# print(system(sprintf("mv *.rds fap_experiments/limitingDepths_duration%f_Gaussian_ntransits10", duration)))

In [ ]:
# zip(zipfile = sprintf('limitingDepths_duration%f_Gaussian_ntransits10.zip', duration), files = sprintf("fap_experiments/faps_duration%f_Gaussian_ntransits10", duration))

In [ ]:
# print(length(list.files(sprintf("fap_experiments/limitingDepths_duration%f_Gaussian_ntransits10", duration))))